In [7]:
import numpy as np

w11 = np.array([-2, -2])
w12 = np.array([2, 2])
w2 = np.array([1, 1])
b1 = 3
b2 = -1
b3 = -1

def MLP(x, w, b):
    y = np.sum(w * x) + b
    if y <= 0:
        return 0
    else:
        return 1

def NAND(x1, x2):
    return MLP(np.array([x1, x2]), w11, b1)

def OR(x1, x2):
    return MLP(np.array([x1, x2]), w12, b2)

def AND(x1, x2):
    return MLP(np.array([x1, x2]), w2, b3)

def XOR(x1, x2):
    return AND(NAND(x1, x2), OR(x1, x2))

if __name__ == '__main__':
    for x in [(0, 0), (1, 0), (0, 1), (1, 1)]:
        y = XOR(x[0], x[1])
        print("입력 값: " + str(x) + "출력 값: " + str(y))

입력 값: (0, 0)출력 값: 0
입력 값: (1, 0)출력 값: 1
입력 값: (0, 1)출력 값: 1
입력 값: (1, 1)출력 값: 0


In [11]:
import tensorflow as tf

x = np.array([[0,0],[0, 1],[1, 0],[1, 1]], dtype = np.float32)
y = np.array([[0], [1],[1], [0]], dtype = np.float32)

w = tf.Variable(tf.random.uniform([2, 1], dtype = np.float32))
b = tf.Variable(tf.random.uniform([1], dtype = np.float32))

def sigmoid(x):
    return 1 / (1+tf.exp(-x))

def perceptron(x):
    return sigmoid(tf.matmul(x, w) + b)

def cost(y_true, y_pred):
    return tf.reduce_mean(-(y_true * tf.math.log(y_pred) + (1 - y_true)*tf.math.log(1 -y_pred)))

optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)

def binary_output(x):
    return int(x.numpy()[0][0] > 0.5)

def accuracy(y_true, y_pred):
    correct_prediction = tf.equal(tf.cast(y_pred > 0.5, tf.float32), y_true)
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for epoch in range(10000):
    with tf.GradientTape() as tape:
        pred = perceptron(x)
        loss = cost(y, pred)
    gradients = tape.gradient(loss, [w,b])
    optimizer.apply_gradients(zip(gradients, [w,b]))

    if (epoch+1) %500 == 0:
        w_val = np.squeeze(w.numpy())
        b_val = np.squeeze(b.numpy())
        print(f"step = {epoch+1}, cost = {loss.numpy():.9f}, w = {w_val}, b = {b_val}")


2025-03-31 21:03:40.043118: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


step = 500, cost = 0.703790307, w = [0.51884794 0.06409978], b = -0.16101372241973877
step = 1000, cost = 0.697289169, w = [0.3622901  0.02921341], b = -0.2066900134086609
step = 1500, cost = 0.695655227, w = [0.2767561  0.03295634], b = -0.18026258051395416
step = 2000, cost = 0.694711685, w = [0.21569085 0.03728088], b = -0.14960385859012604
step = 2500, cost = 0.694134712, w = [0.16913933 0.03859998], b = -0.12316592037677765
step = 3000, cost = 0.693777144, w = [0.13311978 0.03761426], b = -0.10126187652349472
step = 3500, cost = 0.693552732, w = [0.10510145 0.03523072], b = -0.08323230594396591
step = 4000, cost = 0.693410456, w = [0.08322831 0.03211305], b = -0.06840873509645462
step = 4500, cost = 0.693319261, w = [0.06609616 0.02870237], b = -0.05622385814785957
step = 5000, cost = 0.693260431, w = [0.05263427 0.02527872], b = -0.04620874300599098
step = 5500, cost = 0.693221986, w = [0.04202313 0.02201118], b = -0.037977274507284164
step = 6000, cost = 0.693196774, w = [0.0336

In [12]:
print(accuracy(y, pred).numpy())
print(y, pred.numpy())

0.25
[[0.]
 [1.]
 [1.]
 [0.]] [[0.49837536]
 [0.49959484]
 [0.49989524]
 [0.5011148 ]]


In [19]:
import tensorflow as tf
import numpy as np
import datetime

x = np.array([[0,0],[0, 1],[1, 0],[1, 1]], dtype = np.float32)
y = np.array([[0], [1], [1], [0]], dtype = np.float32)

w1 = tf.Variable(tf.random.normal([2, 2]))
b1 = tf.Variable(tf.random.normal([2]))

w2 = tf.Variable(tf.random.normal([2, 2]))
b2 = tf.Variable(tf.random.normal([2]))

w3 = tf.Variable(tf.random.normal([2, 1]))
b3 = tf.Variable(tf.random.normal([1]))

def sigmoid(x):
    return 1 / (1+tf.exp(-x))
@tf.function
def perceptron(x):
    hidden = sigmoid(tf.matmul(x, w1) + b1)
    hidden2 = sigmoid(tf.matmul(hidden, w2) + b2)
    return sigmoid(tf.matmul(hidden2, w3) + b3)
@tf.function
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(-(y_true * tf.math.log(y_pred) + (1 - y_true)*tf.math.log(1 -y_pred)))

optimizer = tf.keras.optimizers.SGD(learning_rate = 0.5)

def binary_output(x):
    return int(x.numpy()[0][0] > 0.5)

def accuracy(y_true, y_pred):
    correct_prediction = tf.equal(tf.cast(y_pred > 0.5, tf.float32), y_true)
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#tensorboard 로그 저장 경로
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = f'logs/mylogs/{stamp}'
writer = tf.summary.create_file_writer(logdir)

#Tensorboard 그래프 추적 및 로그저장
tf.summary.trace_off()
tf.summary.trace_on(graph=True)
pred_initial = perceptron(x)
_ = loss_fn(y, pred_initial)
with writer.as_default():
    tf.summary.trace_export(name="graph_trace", step=0)
print(f"로그저장 -> {logdir}")

for epoch in range(5000):
    with tf.GradientTape() as tape:
        pred = perceptron(x)
        loss = loss_fn(y, pred)
    gradients = tape.gradient(loss, [w1, b1, w2,b2, w3, b3])
    optimizer.apply_gradients(zip(gradients, [w1, b1, w2,b2, w3, b3]))

    acc_val = accuracy(y, pred)

    with writer.as_default():
        tf.summary.scalar("Loss", loss, step=epoch)
        tf.summary.scalar("Accuracy", acc_val, step=epoch)
        
    if (epoch+1) %500 == 0:
        w1_val = np.squeeze(w1.numpy())
        b1_val = np.squeeze(b1.numpy())
        w2_val = np.squeeze(w2.numpy())
        b2_val = np.squeeze(b2.numpy())
        w3_val = np.squeeze(w3.numpy())
        b3_val = np.squeeze(b3.numpy())
        print(f"step = {epoch+1}, cost = {loss.numpy():.9f}")
        print(f"w = {w1_val}, b = {b1_val}")
        print(f"w = {w2_val}, b = {b2_val}")
        print(f"w = {w3_val}, b = {b3_val}")
for input_data in x:
    prediction = perceptron(np.array([input_data], dtype=np.float32))
    print(f"입력: {input_data.tolist()}, 예측(확률): {prediction.numpy()[0][0]:.4f}, 예측(이진): {binary_output(prediction)}")
#최종 정확도
predictions = perceptron(x)
acc = accuracy(y, predictions)
print(f"정확도: {acc.numpy():.4f}")
#수정된 변수명 출력
print(f"TensorBoard 로그 저장 경로: {logdir}")

로그저장 -> logs/mylogs/20250401-191807
step = 500, cost = 0.692169607
w = [[ 1.460471    2.3294432 ]
 [ 0.29151487 -0.92550796]], b = [-0.61677635  0.41012502]
w = [[-0.09294717 -0.24510357]
 [-0.26391256 -0.5222988 ]], b = [-0.7541374  1.9055139]
w = [0.14027849 0.7176617 ], b = -0.6116148233413696
step = 1000, cost = 0.677582502
w = [[ 1.3921965   2.721988  ]
 [ 0.28406507 -1.4733618 ]], b = [-0.6515003   0.94452786]
w = [[-0.0346031  -0.30505243]
 [-0.563315   -1.5373144 ]], b = [-0.7130109  1.4254006]
w = [0.4865203 1.3437825], b = -0.8244594931602478
step = 1500, cost = 0.499317288
w = [[ 1.5147808   4.6680593 ]
 [ 0.10711767 -4.097787  ]], b = [-0.638078   2.4892495]
w = [[ 0.62897825  0.40783736]
 [-1.1553577  -5.0911717 ]], b = [-0.46142778  1.6489611 ]
w = [1.2648015 4.825669 ], b = -1.1267364025115967
step = 2000, cost = 0.038559850
w = [[ 3.5729463  5.621095 ]
 [-3.7078707 -5.129745 ]], b = [-1.8700545  3.1098568]
w = [[ 3.5009565  4.644341 ]
 [-1.8163918 -5.5630426]], b = [-0.

In [37]:
#tensorboard
"""
1. conda info --envs
2. conda activate base
3. tensorboard --logdir=/Users/admin/anaconda_projects/ict_projects/logs/mylogs
4. open http://localhost:6006/
"""

'\n1. conda info --envs\n2. conda activate base\n3. tensorboard --logdir=/Users/admin/anaconda_projects/ict_projects/logs/mylogs\n4. open http://localhost:6006/\n'